In [71]:
from scipy.spatial import distance
import editdistance
import random
from scipy.stats import rankdata
from scipy import stats

In [72]:
'''
Initial chord sets.
'''
keys = ["C","Db","D","Eb","E","F","Gb","G","Ab","A","Bb","B"] #12 semi-tone

maj = dict()
maj7 = dict()
seventh = dict()
minor = dict()
min7 = dict()

for idx in keys:
    maj[idx+":maj"] = list()
    maj7[idx+":maj7"] = list()
    seventh[idx+":7"] = list()
    minor[idx+":min"] = list()
    min7[idx+":min7"] = list()

for root_key in keys:
    majchord = root_key+":maj"
    maj7chord = root_key+":maj7"
    seventhchord = root_key+":7"
    minorchord = root_key+":min"
    min7chord = root_key+":min7"

    rootIdx = keys.index(root_key)

    #maj
    maj[majchord].append(keys[rootIdx])
    maj[majchord].append(keys[(rootIdx+4)%12])
    maj[majchord].append(keys[(rootIdx+7)%12])

    #maj7
    maj7[maj7chord].append(keys[rootIdx])
    maj7[maj7chord].append(keys[(rootIdx+4)%12])
    maj7[maj7chord].append(keys[(rootIdx+7)%12])
    maj7[maj7chord].append(keys[(rootIdx+11)%12])

    #seventh
    seventh[seventhchord].append(keys[rootIdx])
    seventh[seventhchord].append(keys[(rootIdx+4)%12])
    seventh[seventhchord].append(keys[(rootIdx+7)%12])
    seventh[seventhchord].append(keys[(rootIdx+10)%12])

    #minor
    minor[minorchord].append(keys[rootIdx])
    minor[minorchord].append(keys[(rootIdx+3)%12])
    minor[minorchord].append(keys[(rootIdx+7)%12])

    #min7
    min7[min7chord].append(keys[rootIdx])
    min7[min7chord].append(keys[(rootIdx+3)%12])
    min7[min7chord].append(keys[(rootIdx+7)%12])
    min7[min7chord].append(keys[(rootIdx+10)%12])

'''
for Chord Histogram Initail
'''
Chord_Vector = list()
for each_key in keys:
    Chord_Vector.append(each_key+":maj")
    Chord_Vector.append(each_key+":min")
    Chord_Vector.append(each_key+":maj7")
    Chord_Vector.append(each_key+":min7")
    Chord_Vector.append(each_key+":7")



In [73]:
def get_chord_notes_set(chord):

    if "maj7" in chord:
        return maj7[chord]
    elif "min7" in chord:
        return min7[chord]
    elif "maj" in chord:
        return maj[chord]
    elif "min" in chord:
        return minor[chord]
    elif chord=='N' :
        return ['N','N','N']
    else:
        return seventh[chord]

In [74]:
def jaccard_similarity(x,y):
    '''
    Reference:
    Harte, C. (2010).
    Towards automatic extraction of harmony information from music signals
    (Doctoral dissertation, Department of Electronic Engineering, Queen Mary, University of London).
    '''
    
    x = get_chord_notes_set(x)
    y = get_chord_notes_set(y)
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    # I translate similarity to cost

    return (intersection_cardinality/float(union_cardinality))

In [75]:
MAX_COST=1<<32
MAX_COST

4294967296

In [177]:
def LevenshteinDistcance(c1,c2):
    str1=get_chord_notes_set(c1)
    str2=get_chord_notes_set(c2)
    
    str1.sort()
    str2.sort()
    #print('c2:',c2,'str2:',str2)
    m = len(str1)
    n = len(str2)


    #Create a table to store results of subproblems 

    dp = [[MAX_COST for i in range(n+1)] for j in range(m+1)]
    # Fill d[][] in bottom up manner 
    for i in range(0,m+1):

        for j in range(0,n+1): 

            # If first string is empty, only option is to 
            # insert all characters of second string 
            if (i==0):
                dp[i][j] = j  # Min. operations = j 

            # If second string is empty, only option is to 
            # remove all characters of second string 
            elif(j==0):
                dp[i][j] = i # Min. operations = i 

            # If last characters are same, ignore last char 
            # and recur for remaining string 
            elif(str1[i-1] ==str2[j-1]): 
                dp[i][j] = dp[i-1][j-1]

            # If the last character is different, consider all 
            # possibilities and find the minimum 
            else:
                dp[i][j] = 1 + min(dp[i][j-1],  # Insert 
                                   dp[i-1][j],  # Remove 
                                   dp[i-1][j-1]) # Replace 


    
    return dp[m][n]/max(m ,n)


In [77]:
def similarity(c1,c2,option):
    if(option=='jaccard_similarity'):
        return(jaccard_similarity(c1,c2))
    if(option=='LevenshteinDistcance'):
        return(LevenshteinDistcance(c1,c2))

In [170]:
def dtw(sa,sb,similarity_option):
    MAX_COST = 1<<32
    len_sa = len(sa)
    len_sb = len(sb)

    #Initial cost matrix
    dtw_array = [[MAX_COST for i in range(len_sa)] for j in range(len_sb)]

    #dtw_array[0][0] = distance(sa[0],sb[0])
    #dtw_array[0][0] = jaccard_similarity(sa[0],sb[0])
    dtw_array[0][0] = similarity(sa[0],sb[0],similarity_option)
    for i in range(0, len_sb):
        for j in range(0, len_sa):
            if i+j==0:
                continue
            nb = []
            if i > 0: nb.append(dtw_array[i-1][j])
            if j > 0: nb.append(dtw_array[i][j-1])
            if i > 0 and j > 0: nb.append(dtw_array[i-1][j-1])
            min_route = min(nb)
            #cost = distance(sa[j],sb[i])
            cost = similarity(sa[j],sb[i],similarity_option)
            dtw_array[i][j] = cost + min_route
    return dtw_array[len_sb-1][len_sa-1]

In [188]:
import pandas as pd
df = pd.read_pickle('../../../../../data/model_df_cleaned0609')
#df = pd.read_pickle('../../../../../data/model_dfdtw_cleaned0612')


In [95]:
#擁有四張譜以上的歌
#3517筆
has_5ver=df.groupby('song_name').size().loc[lambda x: x>=5].keys()
test_song=has_5ver
df_test=df[df.song_name.isin(test_song)]


len(has_5ver)

541

In [86]:
has_2ver=df.groupby('song_name').size().loc[lambda x: x>=2].keys()
len(has_2ver)

3170

In [189]:
#拿前1000首有兩張譜以上的歌出來當test songs
test_song=has_2ver[:1000]
df_test=df[df.song_name.isin(test_song)]

df_test

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,...,song_name,song_detector_chord,song_chord,chord_before_tf,song_chord_toCkey,star,visitors_0530,tab_dtw_cost,chord_alignment_by_dtw,songchord_alignment_by_dtw
7,76,76,110,5cada0129fc5af34a460af31,ABBA,"[Gb:maj, E:maj, Gb:maj, E:maj, Gb:maj, B:maj, ...",0,29,https://tabs.ultimate-guitar.com/tab/abba/ange...,Angeleyes,...,Angeleyes ABBA,"[{'st': 0, 'et': 7.546, 'ochord': 'N'}, {'st':...","[N, B:maj, Db:maj, B:maj, Gb:maj, B:maj, Ab:ma...","[B:maj, A:maj, B:maj, A:maj, B:maj, E:maj, E:m...","[N, C:maj, D:maj, C:maj, G:maj, C:maj, A:maj, ...",4.5,32,40.8,"[Gb:maj, E:maj, Gb:maj, E:maj, Gb:maj, B:maj, ...","[N, B:maj, Db:maj, B:maj, Gb:maj, B:maj, Ab:ma..."
8,77,77,111,5cada0129fc5af34a460af33,ABBA,"[Db:min, B:maj, Gb:maj, B:maj, Ab:min, E:maj, ...",0,11,https://tabs.ultimate-guitar.com/tab/abba/ange...,Angeleyes,...,Angeleyes ABBA,"[{'st': 0, 'et': 7.546, 'ochord': 'N'}, {'st':...","[N, B:maj, Db:maj, B:maj, Gb:maj, B:maj, Ab:ma...","[Db:min, B:maj, Gb:maj, B:maj, Ab:min, E:maj, ...","[N, C:maj, D:maj, C:maj, G:maj, C:maj, A:maj, ...",5.0,12,21.6,"[Db:min, B:maj, N, N, Gb:maj, B:maj, Ab:min, E...","[N, B:maj, Db:maj, B:maj, Gb:maj, B:maj, Ab:ma..."
16,93,93,127,5cada0159fc5af34a460af63,ABBA,"[G:maj, E:min, G:maj, C:maj, B:maj, A:min, G:m...",0,89,https://tabs.ultimate-guitar.com/tab/abba/does...,Does Your Mother Know,...,Does Your Mother Know ABBA,"[{'st': 0, 'et': 24.776, 'ochord': 'G'}, {'st'...","[G:maj, E:maj, G:maj, C:maj, B:maj, A:min, G:m...","[G:maj, E:min, G:maj, C:maj, B:maj, A:min, G:m...","[C:maj, A:maj, C:maj, F:maj, E:maj, D:min, C:m...",4.5,95,13.1,"[G:maj, E:min, G:maj, C:maj, B:maj, A:min, G:m...","[G:maj, E:maj, G:maj, C:maj, B:maj, A:min, G:m..."
17,94,94,128,5cada0159fc5af34a460af65,ABBA,"[G:maj, G:maj, G:maj, E:min, G:maj, C:maj, G:m...",0,20,https://tabs.ultimate-guitar.com/tab/abba/does...,Does Your Mother Know,...,Does Your Mother Know ABBA,"[{'st': 0, 'et': 24.776, 'ochord': 'G'}, {'st'...","[G:maj, E:maj, G:maj, C:maj, B:maj, A:min, G:m...","[G:maj, G:maj, G:maj, E:min, G:maj, C:maj, G:m...","[C:maj, A:maj, C:maj, F:maj, E:maj, D:min, C:m...",4.5,21,18.3,"[G:maj, G:maj, G:maj, E:min, G:maj, C:maj, G:m...","[G:maj, N, N, E:maj, G:maj, C:maj, B:maj, A:mi..."
19,97,97,131,5cada0159fc5af34a460af6b,ABBA,"[B:maj, E:min, B:maj, A:maj, B:maj, Gb:maj, B:...",0,8,https://tabs.ultimate-guitar.com/tab/abba/eagl...,Eagle,...,Eagle ABBA,"[{'st': 0, 'et': 0.534, 'ochord': 'G#'}, {'st'...","[Ab:maj, E:min, G:maj, A:maj, E:maj, C:maj, E:...","[E:min, A:min, E:min, D:maj, E:min, B:maj, E:m...","[N, D:min, F:maj, G:maj, D:maj, N, D:min, G:ma...",4.0,8,43.9,"[B:maj, E:min, N, N, N, N, N, N, N, N, N, N, N...","[Ab:maj, E:min, G:maj, A:maj, E:maj, C:maj, E:..."
20,98,98,132,5cada0169fc5af34a460af6d,ABBA,"[E:min, E:min, B:maj, E:min, A:maj, E:min, A:m...",0,19,https://tabs.ultimate-guitar.com/tab/abba/eagl...,Eagle,...,Eagle ABBA,"[{'st': 0, 'et': 0.534, 'ochord': 'G#'}, {'st'...","[Ab:maj, E:min, G:maj, A:maj, E:maj, C:maj, E:...","[E:min, E:min, B:maj, E:min, A:maj, E:min, A:m...","[N, D:min, F:maj, G:maj, D:maj, N, D:min, G:ma...",5.0,21,32.3,"[E:min, E:min, B:maj, E:min, A:maj, E:min, N, ...","[Ab:maj, E:min, N, G:maj, A:maj, E:maj, C:maj,..."
22,101,101,135,5cada0169fc5af34a460af79,ABBA,"[E:maj, A:maj, A:maj, B:maj, A:maj, A:maj, B:m...",0,22,https://tabs.ultimate-guitar.com/tab/abba/fern...,Fernando,...,Fernando ABBA,"[{'st': 0, 'et': 1.231, 'ochord': 'N'}, {'st':...","[N, A:maj, D:maj, C:maj, A:maj, E:maj, Gb:maj,...","[A:maj, D:maj, D:maj, E:maj, D:maj, D:maj, E:m...","[N, F:maj, N, N, F:maj, C:maj, D:maj, F:maj, G...",5.0,23,33.1,"[E:maj, A:maj, A:maj, B:maj, A:maj, A:maj, B:m...","[N, A:maj, N, D:maj, C:maj, A:maj, E:maj, N, N..."
23,102,102,136,5cada0169fc5af34a460af7b,ABBA,"[A:maj, Gb:min, B:maj, E:maj, A:maj, A:maj, Gb...",0,233,https://tabs.ultimate-guitar.com/tab/abba/fern...,Fernando,...,Fernando ABBA,"[{'st': 0, 'et': 1.2

In [179]:
LevenshteinDistcance('Db:maj','F:min')

0.3333333333333333

In [196]:
std=['C:maj','G:maj','A:min','E:min','F:maj']
b0=['C:maj','G:maj','A:min','E:min','F:maj']

1-dtw(std,b0,'LevenshteinDistcance')/max(len(std),len(b0))
#1-dtw(std,b0,'jaccard_similarity')/max(len(std),len(b0))
#1-dtw_table(std,b0,'jaccard_similarity')/max(len(std),len(b0))

1.0

In [172]:
def dtw_table(x, y,similarity_option):
    nx = len(x)
    ny = len(y)
    table = np.zeros((nx+1, ny+1))
    
    # Compute left column separately, i.e. j=0.
    table[1:, 0] = np.inf
        
    # Compute top row separately, i.e. i=0.
    table[0, 1:] = np.inf
        
    # Fill in the rest.
    for i in range(1, nx+1):
        for j in range(1, ny+1):
            #d = jaccard_similarity(x[i-1], y[j-1])
            d = similarity(x[i-1], y[j-1],similarity_option)
            table[i, j] = d + min(table[i-1, j], table[i, j-1], table[i-1, j-1])
    return table[i,j]

sa = ["C:maj","F:maj","G:maj","C:maj"]
sb = ["C:maj","F:maj","C:maj"] #delete C:maj
table=dtw_table(sa,sb,'jaccard_similarity')
table

2.2

In [35]:
#funtion:dtw
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    dtw_cost=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_cost.append(dtw(std,b0))
    min_dtw_cost_index=np.argmin(dtw_cost)
    if(min_dtw_cost_index==0):
        c_1 +=1
 

 

top1 = c_1 / float(1000)



print('Top-1 Precision: %f' % top1)




0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.427000


In [39]:
#similarity funtion:1-dtw(c1,c2,jaccard_similarity)/max(len(c1),len(c2)) 
#找similarity最高的譜
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    dtw_cost=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_cost.append(1-dtw(std,b0,'jaccard_similarity')/max(len(std),len(b0)))
    min_dtw_cost_index=np.argmax(dtw_cost)
    if(min_dtw_cost_index==0):
        c_1 +=1
 

 

top1 = c_1 / float(1000)



print('Top-1 Precision: %f' % top1)



0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.435000


In [198]:
#similarity funtion:1-DTW(cq1,cq2,LevenshteinDistcance)/max(len(cq1),len(cq2)) 
#找最高similarity的譜
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
top_n=[0,0,0,0]
tau_k=[]
p_value_k=[]
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)
    rating=df_tmp.rating
    std=df_tmp.iloc[0].song_chord
    dtw_sim=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_sim.append(1-dtw(std,b0,'LevenshteinDistcance')/max(len(std),len(b0)))
    max_dtw_sim_index=np.argmax(dtw_sim)
    
    sim_r = rankdata(dtw_sim, method='max')
    rating_r=rankdata(rating,method='max')


    tau,p_value=stats.kendalltau(sim_r, rating_r)
    
    if tau is not np.nan:
                
        tau_k.append(tau)
    if p_value is not np.nan:
                
        p_value_k.append(p_value)
    
    #Top-n precision:
    if(max_dtw_sim_index==0):
        c_1 +=1
        top_n[0] +=1
#     for top_n_number in range(1,4):
#         if np.argmax(dtw_sim)<=top_n_number:top_n[top_n_number] +=1
        

top1 = c_1 / float(len(test_song))



print('Top-1 Precision: %f' % top1)
# for top_n_number in range(0,4):
#     print('Top-n Precision: %f' % (top_n[top_n_number]/float(len(test_song))),'n=',top_n_number+1)
print('kendall-tau:',sum(tau_k)/len(tau_k))
print('kendall-tau pvalue:',sum(p_value_k)/len(p_value_k))

0


C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.439000
kendall-tau: 0.09392735311117414
kendall-tau pvalue: nan


In [84]:
print('Top-1 Precision: %f' % top1)

Top-1 Precision: 0.424000


In [70]:
#similarity funtion:1-DTW(cq1,cq2,jaccard_similarity)/max(len(cq1),len(cq2)) 
#找最高similarity的譜
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
top_n=[0,0,0,0]
tau_k=[]
p_value_k=[]
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)
    rating=df_tmp.rating
    std=df_tmp.iloc[0].song_chord
    dtw_sim=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_sim.append(1-dtw(std,b0,'jaccard_similarity')/max(len(std),len(b0)))
    max_dtw_sim_index=np.argmax(dtw_sim)
    
    sim_r = rankdata(dtw_sim, method='max')
    rating_r=rankdata(rating,method='max')


    tau,p_value=stats.kendalltau(sim_r, rating_r)
    
    if tau is not np.nan:
                
        tau_k.append(tau)
    if p_value is not np.nan:
                
        p_value_k.append(p_value)
    
    #Top-n precision:
    if(max_dtw_sim_index==0):
        c_1 +=1
        top_n[0] +=1
#     for top_n_number in range(1,4):
#         if np.argmax(dtw_sim)<=top_n_number:top_n[top_n_number] +=1
        

top1 = c_1 / float(len(test_song))



print('Top-1 Precision: %f' % top1)
# for top_n_number in range(0,4):
#     print('Top-n Precision: %f' % (top_n[top_n_number]/float(len(test_song))),'n=',top_n_number+1)
print('kendall-tau:',sum(tau_k)/len(tau_k))
print('kendall-tau pvalue:',sum(p_value_k)/len(p_value_k))

0


C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.392000
kendall-tau: -0.014184606067471081
kendall-tau pvalue: nan


In [211]:
#funtion:dtw_table
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    dtw_cost=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        dtw_cost.append(dtw(std,b0,'jaccard_similarity'))
    min_dtw_cost_index=np.argmin(dtw_cost)
    if(min_dtw_cost_index==0):
        c_1 +=1
 

 

top1 = c_1 / float(1000)



print('Top-1 Precision: %f' % top1)


0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.409000


# Chord  Histogram算歌曲相似度


In [218]:
chord_index={
    'C:maj':  1,
    'Db:maj': 2,
    'Db:min': 3,
    'D:maj':  4,
    'Eb:maj': 5,
    'Eb:min': 6,
    'E:maj':  7,
    'E:min':  8,
    'F:maj':  9,
    'F:min':  10,
    'Gb:maj': 11,
    'Gb:min': 12,
    'G:maj':  13,
    'G:min':  14,
    'Ab:maj': 15,
    'Ab:min': 16,
    'A:maj':  17,
    'A:min':  18,
    'Bb:maj': 19,
    'Bb:min': 20,
    'B:maj':  21,
    'B:min':  22    

}
chord_list=['C:maj',
            'Db:maj',
            'Db:min',
            'D:maj',
            'Eb:maj',
            'Eb:min',
            'E:maj',
            'E:min',
            'F:maj',
            'F:min',
            'Gb:maj',
            'Gb:min',
            'G:maj',
            'G:min',
            'Ab:maj',
            'Ab:min',
            'A:maj',
            'A:min',
            'Bb:maj',
            'Bb:min',
            'B:maj',
            'B:min'
           ]


22

In [222]:
from scipy.spatial import distance
def chord_hist_L2dist(cq1,cq2):
    a=[]
    b=[]
    for chord in chord_list:
        a.append(cq1.count(chord)/len(cq1))
        b.append(cq2.count(chord)/len(cq2))
    return distance.euclidean(a, b)

In [236]:

#funtion:chord_hist_L2dist
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    cq_dist=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord
        cq_dist.append(chord_hist_L2dist(std,b0))
    
    min_cq_dist_index=np.argmin(cq_dist)
    
    if(min_cq_dist_index==0):
        c_1 +=1

top1 = c_1 / float(len(test_song))



print('Top-1 Precision: %f' % top1)


0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.409000


In [235]:
#funtion:chord_hist_L2dist
top1_ls = []
mrr_ls = []
c_1, c_2 = 0, 0
#for each test song
for i,song_name in enumerate(test_song):

    if(i %100==0):print(i)

    df_tmp=df_test[df_test.song_name==song_name]
    df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

    std=df_tmp.iloc[0].song_chord
    cq_dist=[]
    for index, row in df_tmp.iterrows():
        b0=row.chord_before_tf
        cq_dist.append(chord_hist_L2dist(std,b0))
    
    min_cq_dist_index=np.argmin(cq_dist)
    
    if(min_cq_dist_index==0):
        c_1 +=1

top1 = c_1 / float(len(test_song))



print('Top-1 Precision: %f' % top1)


0
100
200
300
400
500
600
700
800
900
Top-1 Precision: 0.388000


0